## 네이버 카페 스크랩


## 필요한 라이브러리 설치
* 아나콘다 사용시 다음의 프롬프트 창을 열어 conda 명령어로 설치합니다.
* pip 사용시 아래에 있는 명령어를 터미널로 설치합니다.
<img src="https://i.imgur.com/Sar4gdw.jpg">

### Selenium
* `conda install -c anaconda selenium`
* [Selenium :: Anaconda Cloud](https://anaconda.org/anaconda/selenium)

* pip 사용시 : `pip install selenium`

### BeautifulSoup
* `conda install -c anaconda beautifulsoup4`
* [Beautifulsoup4 :: Anaconda Cloud](https://anaconda.org/anaconda/beautifulsoup4)

* pip 사용시 : `pip install beautifulsoup4`

### tqdm
* `conda install -c conda-forge tqdm`
* [tqdm/tqdm: A Fast, Extensible Progress Bar for Python and CLI](https://github.com/tqdm/tqdm)
* `pip install tqdm`

%pip install pyperclip

In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd
from pandas import read_excel
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import pyperclip

import time
from tqdm import trange
from tqdm import tqdm

options = webdriver.ChromeOptions()
options.headless = True
options.add_argument("window-size=1920x1080") # 가상화면 크기
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36")

# 브라우저 기동
browser = webdriver.Chrome(options=options)

# 네이버 카페 로그인
#클립보드에 input을 복사한 뒤
#해당 내용을 actionChain을 이용해 로그인 폼에 붙여넣기
def copy_input(xpath, input):
    pyperclip.copy(input)
    browser.find_element_by_xpath(xpath).click()
    ActionChains(browser).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)

id = 'issac_jung'
pw = '2020@Alpha'

browser.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')
browser.implicitly_wait(3)

copy_input('//*[@id="id"]', id)
time.sleep(1)
copy_input('//*[@id="pw"]', pw)
time.sleep(1)
browser.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

keywords = ['외식', '식사', '메뉴', '식당', '레스토랑', '가성비', '트렌드', '배달', '맛집', '비대면']
keywords = ['식사', '메뉴', '식당', '레스토랑', '가성비', '트렌드', '배달', '맛집', '비대면']

# print(keywords)
# keywords = [ '1인 외식']

# 페이지 로딩 대기 시간(초)
interval = 3

# 다음뉴스 키워드 검색 Main 호출
pnum = 1000 # 1500개 news 가져오기, 15 페이지 (페이지당 10개 )

# 키워드 별 URL 파일 읽어 blog post 가져오기
for keyword in keywords:

    # 파일에서 읽어 원하는 모양으로 변환 (모두 List 형)으로 변환
#     urls_filename = "./urls/" + "naver-cafe_" + keyword.replace(" ","") + ".csv"
#     df_postURLs = pd.read_csv(urls_filename, header=1)
#     urls = df_postURLs.values.tolist()

    titles = []
    dates = []
    counts = []
    contents = []
    post_df = pd.DataFrame(columns=("title", "date", "count", "content"))
    
    start_date = "2019.07.01"
    end_date = "2020.06.30"
    
    pages = (pnum // 10 ) + 1
    for page in trange(1, pages):  # 150 번 페이지

        url = f'https://m.cafe.naver.com/SectionArticleSearch.nhn?page={page}&sortBy=0&query={keyword}&period=["2019.07.01","2020.06.30"]'
#         url = f'https://section.cafe.naver.com/cafe-home/search/articles?query={keyword}

        for i in range(0, 10):
            try:
                browser.get(url)
                time.sleep(1)
                browser.find_elements_by_css_selector("strong.tit")[i].click()
                time.sleep(1)
                          
                post_soup = bs(browser.page_source, 'html.parser')  # 웹 페이지 파싱

                title = post_soup.find('h2', attrs={"class":"tit"}).get_text().strip()
                date = post_soup.find('span', attrs={"class":"date font_l"}).get_text().strip()
                count = post_soup.find('span', attrs={"class":"no font_l"}).get_text().strip()
#                 content = post_soup.find('div', attrs={"class":"se-main-container"}).get_text().strip()
                content = post_soup.find('div', attrs={"class":"post_cont font_zoom1"}).get_text().strip()

                titles.append(title)
                dates.append(date)
                counts.append(count)
                contents.append(content)
            
            except:
#                 print("오류 발생")
                continue

    post_df = pd.DataFrame({"title": titles, "date": dates, "count": counts, "content": contents})
#     print(post_df)

    # 파일 쓰기
    output_filename = "./scraps/" + "naver-cafe-scrapped_" + keyword.replace(" ","") + ".csv"
#     print(output_filename)

    post_df.to_csv(output_filename, date_format='%Y%m%d', encoding='utf-8-sig')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [51:59<00:00, 31.20s/it]
